## Storm Prediction Center thermodynamic and upper air maps

### Justin Richling  4/17/2017

### http://www.spc.noaa.gov/exper/mesoanalysis/

### I use these maps to try and track storms to see what kind of data they are producing. SPC has an almost endless array of maps that provide so much valuable information as well as predictions. They can be overlapped and/or compared with others to get a good detail of what's happeningin the air!

## Updates every hour

In [11]:
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import time,urllib,urllib2,cStringIO,logging,webbrowser,\
IPython.display,shutil,sys,os
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from IPython.core.display import Image
from PIL import Image as PILImage
from matplotlib.pyplot import *

In [2]:
from datetime import datetime
datetime.utcnow().hour
#now = datetime.now()

16

In [3]:
ListRegion = ["National","Mid West","Southwest","Southern Plains","Northern Plains",\
"Northeast","Southeast","Central Plains","Northwest","East Coast"]

List = []
WinterList = []
RegionNumber = [19,20,12,15,13,16,18,14,11,17]

# This will litterally match each entry of both lists as an entry for a new list
Regionz = zip(ListRegion,RegionNumber)

# These are the extensions needed for SPC's webpage
Mapext = ["tran","pwtr","cpsh","3cvr","eshr","lfrh","lclh","sbcp","300mb","500mb","700mb",\
"850mb","925mb","ttd","rgnlrad","1kmv","bigsfc","pmsl","vtm","stor","laps"]

# This list is just a quick description of each map, this will serve as axis titles info
Mapname = ["850mb Precip Transfer","Precipital Water, Lowest 400mb","MUCAPE, MUCIL, Effective Bulk Shear",\
"0-3 km MLCAPE and Surface Vorticity","Effective Bulk Shear","LCL to LFC Mean RH",\
"100mb Mean Parcel LCL","Surface Based CAPE/CIN","300mb","500mb","700mb",\
"850mb","925mb","Surface Temp, Dew Point, and Mean Sea Level Pressure Isobars","RADAR",\
"Visible Satellite","Surface Observations","MSLP and Winds","Horz. Velocity Gradient",\
"Sig. Tornado Parameter","700-500mb Lapse Rate"]

winterMapext = ["fztp","swbt","fzlv","thck","dend"]

winterMapname = ["Near Freezing Surf Temps","Near Freezing Wet Bulb Temp","Freezing Level",\
"Critical Thickness","Dendritic Layer Depth"]

# Now we can populate the new list, List, with the full address of each map.
# This will ais in retrieving the all the maps the user desires
for i in RegionNumber:
    #print i
    for j in Mapext:
        List.append("http://www.spc.noaa.gov/exper/mesoanalysis/s"+str(i)+"/"+str(j)+"/"+str(j)+".gif")
        
for i in RegionNumber:
    #print i
    for j in winterMapext:
        WinterList.append("http://www.spc.noaa.gov/exper/mesoanalysis/s"+str(i)+"/"+str(j)+"/"+str(j)+".gif")

In [4]:
# (region, SPC's region number)
Regionz

[('National', 19),
 ('Mid West', 20),
 ('Southwest', 12),
 ('Southern Plains', 15),
 ('Northern Plains', 13),
 ('Northeast', 16),
 ('Southeast', 18),
 ('Central Plains', 14),
 ('Northwest', 11),
 ('East Coast', 17)]

In [5]:
Nat = List[0:18]
MW = List[18:36]
SW = List[36:54]
SPl = List[54:72]
NPl = List[72:90]
NE = List[90:108]
SE = List[108:126]
CPl = List[126:144]
NW = List[144:162]
EC = List[162:180]
# Regions will be the final array of all the maps for each region, 18 maps x 10 regions = 
#180 maps in all right now!
Regions = np.array((Nat,MW,SW,SPl,NPl,NE,SE,CPl,NW,EC))

In [6]:
# mappage is the full web address for the particular map
# maptitle will be not only the extension on the saved image file name, but also the 
#    title to the image itself
# save_location will be where the new image will be saved, providyed by user

def SPC_Storm_Pages(year,month,day,hour,minute,mappage,maptitle,Region,save_location):
    
    if month < 10:
        month = "0"+str(month)
    if day < 10:
        day = "0"+str(day)
    if hour < 10:
        hour = "0"+str(hour) 

    try:
        print mappage

# Open a browser to the page
    #webbrowser.open(picture_page) 

# open the web page picture and read it into a variable
        opener1 = urllib2.build_opener()
        page1 = opener1.open(mappage)
        my_picture = page1.read()
            
# open file for binary write and save picture
# picture_page[-4:] extracts extension eg. .gif
# (most image file extensions have three letters, otherwise modify)
        Filename = str(year)+"_"+str(month)+"_"+str(day)+\
    "_"+str(hour)+"_"+str(minute)+"Z"+"_"+str(maptitle)+".png" 
        fout = open(save_location+"/"+\
                    Filename, "wb")
        fout.write(my_picture)
        fout.close()
          
# test to see if it saved correctly
        #webbrowser.open(Filename)                   
    except urllib2.HTTPError, err:
        if err.code == 404:
            logging.exception("No Data from website")
            pass
        else:
            raise

    file = cStringIO.StringIO(urllib.urlopen(mappage).read())
    img = PILImage.open(file)
    fig, ax = subplots(figsize=(15, 8))
    ax.set_title(str(maptitle)+str(year)+" "+\
                 str(month)+" "+str(day)+" "+str(hour)+str(minute)+"Z")
    ax.imshow(img, interpolation='nearest')
    tight_layout()
    #plt.show()
    plt.close()

In [27]:
def RunMap(Year,Month,Day,Hour,Minute,j,region,save_location):
    # The kth index will be used for seperating map titles in NOAA's website
    k = 0   
    
    # The jth index of Regions is which region of NOAA's map we're looking at
    for i in Regions[j]:
    # Inputs and necessary position in the function:
        # Mapext[k] will grab each of the extensions for the maps needed for the webpage
        # i will iterate through the regions provided by user
        SPC_Storm_Pages(Year,Month,Day,Hour,Minute,i,Mapext[k],region,save_location)
    # This will step k to the next int by 1, thus getting all the map names    
        k = k+1

In [ ]:
# Keeps track of how many regions the user wants to track
Inputz = raw_input("How many regions would you like to track (10 total)? (A)ll to run all regions ") 

# MyInput is a new list to accept all the regions the user inputs
MyInput = []

if Inputz == "A":
    # prints all regions
    for item in Regionz:
        print item
else:
    Inputz = int(Inputz)  # need to make the number provided an actual integer
    for i in range(Inputz):
    # Give a list of region names to be used as input by user
        for h in range(len(ListRegion)):
            print str(ListRegion[h])
        MyInput.append(raw_input("Which Region: "))
        print "\n"+"Regions for Loop: "+str(MyInput)+"\n"

# Number of hours desired by user for loop        
Time = input("How many hours for loop? ")
print
# datetime.now gives us current time from operating system; now stores it in a variable
now = datetime.now()
Month = now.month
Year = now.year
Day = now.day

# utcnow will give us 24 hr clock. This helps because we use UTC so much already    
Hour = datetime.utcnow().hour
Minute = datetime.utcnow().minute

# We need the month and day number to have a 0 in front for the web address
# This will just concoctinate?? add a zero if its less than 10     
if Month < 10:
    Month = "0"+str(Month)
if Day < 10:
    Day = "0"+str(Day)
if Hour < 10:
    Hour = "0"+str(Hour)
if Minute < 10:
    Minute = "0"+str(Minute)
    
# We want to store this particular storm in its designated folder 
# Each region will have its own folder, but this is where all the region folders
# will be saved
save_location = raw_input("Where would you like to save the folders? ")

for g in range(Time*6):
    for i in range(len(MyInput)):    
    # Check to see if Input (user's input) is in our list of NOAA regions
        if MyInput[i] in ListRegion:
    # If Input is in our list, then set j equal to that numbered index
    # j will be an int!!
            j = ListRegion.index(MyInput[i])
    # We will use j in a moment, it is the list entry number of input region
        
        # The new save location will just add the date to it so each region will be saved as a new
        # variable/folder (save_location_new) in the original save location (save_location) above 
        save_location_new = str(save_location)+str(Year)+"_"+str(Month)+"_"+str(Day)+"_"+str(MyInput[i])+"/"
        #print "Directory to house all region folders: "+str(save_location)
        #print "new save_location: "+str(save_location_new)+"\n"

# Checking to see if the folder exists already (os.path.isdir) and if not, create one (os.makedirs)
# Then redirect (os.chdir) to that folder so later the created image can be automatically saved to it
        if not os.path.isdir(save_location_new):
            os.makedirs(save_location_new)
        os.chdir(save_location_new)
        print str(MyInput[i])+":  "+str(save_location_new)
        RunMap(Year,Month,Day,Hour,Minute,j,MyInput[i],save_location_new)
        print
        
    # The time.sleep() at the end of the loop will allow for the time desired by SPC to
    # regenerate an updated map. I believe its 10 or 20 min for SPC maps??
    time.sleep(10*60)
    
print "All done."